In [1]:
import pandas as pd

Combine data with Core CPI, Unemployment rate, Stock Markets with data with GDP in US\$ for each country

First data comes in a monthly basis, second data comes in a quarterly basis.

In [75]:
# define a function that combines two data
def combine(m_data, q_data, country_name):
    # format date and use as index
    m_index = m_data['index'].apply(lambda x: x[:7])
    m_date = pd.to_datetime(m_index, format = '%YM%m')
    m_data.loc[:,'index'] = m_date
    # format date and use as index
    q_index = q_data['index'].apply(lambda x: x.replace('Q', 'M')[:6])
    q_index = q_index.apply(lambda x: x[:-1] + str(int(x[-1])*3-2))
    q_date = pd.to_datetime(q_index, format = '%YM%m')
    q_data.loc[:,'index'] = q_date
    # merge two data, forward fill the missing value
    combined = m_data.merge(q_data, on = 'index', how = 'left')
    combined.fillna(method='ffill', inplace = True)
    # add country name into the data
    combined['country'] = country_name
    
    return combined

In [76]:
# save all country names in a list
country_names = ['Brazil', 'Canada', 'HongKong', 'Japan', 'Mexico',
                'Russia', 'SouthAfrica', 'Thailand', 'UnitedKingdom',
                'UnitedStates']

In [86]:
# define a dataframe to save combined data
GDP_data = pd.DataFrame(columns= ['index', 'Core CPI', 'Unemployment rate', 
                                  'Stock Markets', 'GDP in US$', 'country'])

In [87]:
# combine data for each country in the country_name list
# and append to the dataframe
for country in country_names:
    m_path = './EDI_dataset/' + country + '_M.csv'
    q_path = './EDI_dataset/' + country + '_Q.csv'
    
    m_data = pd.read_csv(m_path)
    q_data = pd.read_csv(q_path)
    
    combined_data = combine(m_data, q_data, country)
    
    GDP_data =  GDP_data.append(combined_data)

In [99]:
# remove redundant columns
GDP_data.drop(['GDP in US$', 'level_0_x', 'level_0_y'], axis = 1, inplace = True)

In [119]:
# show dataframe
GDP_data

,index,Core CPI,Unemployment rate,Stock Markets,country,GDP in US$
0,2008-01-01,90.153723,11.157871,87.101121,Brazil,3662800.0
1,2008-02-01,90.694644,11.312779,94.025339,Brazil,3662800.0
2,2008-03-01,91.111840,10.791905,93.490682,Brazil,3662800.0
3,2008-04-01,91.631178,10.700540,98.750100,Brazil,3701400.0
4,2008-05-01,92.217617,9.939990,111.320716,Brazil,3701400.0
...,...,...,...,...,...,...
146,2020-03-01,121.457975,4.400000,216.186728,UnitedStates,5384900.0
147,2020-04-01,120.902284,14.700000,222.456085,UnitedStates,5384900.0
148,2020-05-01,120.770517,13.300000,231.787567,UnitedStates,5384900.0
149,2020-06-01,120.999064,11.100000,248.893940,UnitedStates,5384900.0


In [101]:
# export data to csv file to be used to calcualte EDI
GDP_data.to_csv('combined_indicators.csv')

The csv file 'combined_indicators.csv' will be used by another team member to calculate EDI. The method deployed for this calculation is PCA. Note that filename is changed to 'all_indicators.csv' on the other end.

Next look at the range of predictor variable. We need to find the overlapped time window between the response and predictors.

In [104]:
# import predictors
final_data = pd.read_csv('./final_data.csv')

In [114]:
# check predictors' starting data
final_data['date'].unique()[0]

'2020-02-15'

In [112]:
# check predictor's ending date
final_data['date'].unique()[-1]

'2020-10-27'

After response variable - EDI is computed for each country, import this data.

Because the EDI data comes in a monthly basis, and the predictor data was collected for everday between 2020-02-15 to 2020-10-27, we need to do linear interpolation on the EDI data. Note that we will start from 2020-02-01 isntead of 2020-02-15 because the EDA data is available on the first day of every month, so we will need to use 2020-02-01 and 2020-03-01 to interpolate the EDI value for every day in February. After interpolation, we will discard interpolated EDI data before 2020-02-15 so that we will only be left with the overlapped data between response and predictors.

In [120]:
# read computed EDI data
edi = pd.read_csv('./EDI_indicators.csv')

In [126]:
# create a time index to work with
time_index = pd.date_range(start = '2/1/2020', end = '7/1/2020')

In [166]:
# create a dataframe to store interpolated data
interpolated_df = pd.DataFrame(columns=['country', 'EDI'])

In [169]:
# loop through all countries and perform interpolation
for country in country_names:
    # subset edi data for each country
    temp_df = edi.loc[edi['country'] == country, ]
    # use date as index
    temp_df.set_index('index', inplace = True)
    temp_df.index = pd.to_datetime(temp_df.index)
    temp_df = temp_df.reindex(time_index)
    # perform interpolation and forward fill missing values
    temp_df = temp_df.interpolate().fillna(method = 'ffill')
    temp_df = temp_df.reset_index()
    
    interpolated_df = interpolated_df.append(temp_df)

In [176]:
# change names for consistence for future use
interpolated_df.replace({'UnitedKingdom':'United Kingdom',
                        'UnitedStates':'Untied States',
                        'SouthAfrica':'South Africa'}, inplace = True)

In [179]:
# drop EDI data for hongkong
interpolated_df = interpolated_df.loc[interpolated_df['country'] != 'HongKong', ]

In [181]:
# export data as csv
interpolated_df.to_csv('interpolated_edi.csv')